In [1]:
# import get_factor_tools as gft
import datetime
import time
import sys

import pymysql
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import multiprocessing
import os
import queue

In [2]:
data = pd.read_csv("/home/hengtian/kate/ideal/ideal_chart.csv", index_col=0)

/home/hengtian/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
data

In [3]:
avg = pd.pivot_table(data, values='avg_deal_amount', index='date', columns='stock_code',dropna=True).sort_index()
# amount = pd.DataFrame(,index=value.index, columns = stocks)

In [4]:
chg =pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/close_price_post.csv", index_col = 0)

In [5]:
chg.index = pd.to_datetime(chg.index)

In [6]:
n_pct = chg/chg.shift(1)-1
n_pct.index = pd.to_datetime(n_pct.index)
n_pct = n_pct['2016':'2021-06-22']
colum = [col for col in avg.columns if col in list(n_pct.columns)]
n_pct = n_pct[colum]
avg = avg[colum]
avg.index = pd.to_datetime(avg.index)

In [25]:
avg

stock_code,000001.XSHE,000002.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,000011.XSHE,...,688689.XSHG,688696.XSHG,688698.XSHG,688699.XSHG,688700.XSHG,688777.XSHG,688788.XSHG,688819.XSHG,688981.XSHG,689009.XSHG
date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,26590.543713,NaN,30619.080176,10501.406287,27289.653279,NaN,11805.609274,24541.513514,16703.453674,16585.833580,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-05,25859.306353,NaN,31859.236286,12550.701164,24224.555239,NaN,12970.640847,22432.944637,15847.859092,16690.392597,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-06,29063.240845,NaN,24103.908865,12131.620543,22500.059579,NaN,12265.698308,21889.277342,16171.376315,16046.698625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-07,29543.586032,NaN,22813.142265,12532.283926,20926.078092,NaN,11340.269118,24799.561412,14621.762534,15354.471087,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-08,28664.731613,NaN,22534.625056,11036.418772,22490.354718,NaN,12741.847488,23911.132660,15134.646877,16375.175118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-16,18371.517120,24224.218302,13838.186313,9392.313426,7544.395070,22357.516006,8097.191835,21695.041135,9073.160579,9585.095148,...,17722.556998,129521.335034,10493.688980,51570.737733,31475.952312,30251.122026,36930.198630,24808.711396,36921.971361,35543.284913
2021-06-17,20014.017731,25006.191250,13217.149611,9278.642399,8493.621917,26276.716285,7826.766118,19350.797652,8466.654253,11549.371600,...,22791.502922,133168.817991,9875.947843,61649.234215,35745.529469,34466.622625,39136.220793,24228.890056,40665.169418,34732.538767
2021-06-18,23267.781893,21922.199051,12016.771808,11434.774317,8971.109355,17729.295651,8210.492805,17813.926155,5578.322159,11827.625130,...,21376.294333,148041.132277,9951.051181,55502.018220,37560.595544,30781.952358,37174.385516,27751.496846,44739.155349,41396.728354


In [7]:
def task(stock,q):
    i=20
    s_avg = avg[stock]
#     print(s_avg)
    s_pct = n_pct[stock]
    s_m=np.full(len(n_pct), np.nan)
    s_h = np.full(len(n_pct), np.nan)
    s_l=np.full(len(n_pct), np.nan)
    while i<len(n_pct):
        if len(s_avg.iloc[i-20:i].dropna())!= 20:
            i+=1
        else:
            ss_avg = s_avg.iloc[i-20:i]
            ss_pct=s_pct.iloc[i-20:i]
            med = ss_avg.median()
            high = ss_avg[ss_avg>med].index
#             print(high)
            low =ss_avg[ss_avg<=med].index
            m_high = ss_pct.loc[high].sum()
#             s_h[i] = m_high
            m_low=ss_pct.loc[low].sum()
#             s_l[i]=m_low
#             print(ss_avg, med, high)
            s_m[i] = m_high-m_low
            i+=1
#     list0 =np.concatenate((s_l,s_h),axis = 1)
#     list1 = np.concatenate((np.array(list0),s_m),axis=1)
    return q.put({stock:s_m})
      
# test = task('000001.XSHE')
# x = np.array(test)
# x.shape

In [8]:
# d = pd.DataFrame(np.empty((len(avg),avg.shape[1]),dtype=object),columns = avg.columns, index = avg.index)
# d_l = pd.DataFrame(np.empty((len(avg),avg.shape[1]),dtype=object),columns = avg.columns, index = avg.index)
# d_h = pd.DataFrame(np.empty((len(avg),avg.shape[1]),dtype=object),columns = avg.columns, index = avg.index)

q = multiprocessing.Manager().Queue()
param = []
for stock in avg.columns:
    param.append((stock,q))
# print(param)
#task
with multiprocessing.Pool(processes =4) as pool:
    pool.starmap(task,param)

#extract
elements = []
while q.qsize():
    elements.append(q.get())

# print(elements)
    #store in stock
   

In [9]:
d = pd.DataFrame(np.empty((len(avg),avg.shape[1]),dtype=object),columns = avg.columns, index = avg.index)
for i in range(len(elements)):
    stock = list(elements[i].keys())[0]
    val = list(elements[i][stock])
    d[stock]=val

# if len(s_m)<len(avg):
#     l = np.zeros(len(avg)-len(s_m)).tolist()
#     s_m = s_m+l
# d[stock] =s_m.transpose()
# d_l[stock]=s_l.transpose()
# d_h[stock]=s_h.transpose()

In [10]:
d.to_csv("ideal_final.csv")

In [11]:
d

stock_code,000001.XSHE,000002.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,000011.XSHE,...,688689.XSHG,688696.XSHG,688698.XSHG,688699.XSHG,688700.XSHG,688777.XSHG,688788.XSHG,688819.XSHG,688981.XSHG,689009.XSHG
date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-16,0.021016,0.160976,0.351013,0.087523,-0.020616,-0.031238,-0.026794,0.486167,-0.057138,0.252412,...,0.148980,0.148684,-0.155020,-0.159685,NaN,-0.069743,-0.042043,0.162469,0.119097,0.105388
2021-06-17,0.032005,0.162785,0.278802,0.110311,-0.013180,-0.031716,-0.035645,0.381993,-0.062233,0.287473,...,0.133014,0.198078,-0.102151,-0.132019,NaN,-0.032330,0.037968,0.223073,0.116554,0.062773
2021-06-18,0.026946,0.136081,0.230782,0.087266,0.003717,-0.006661,-0.048825,0.532570,-0.064939,0.253348,...,0.349775,0.128566,-0.126960,0.033139,NaN,-0.004591,0.060332,0.217715,0.169455,0.191050


In [ ]:
df = pd.read_csv("i_chart.csv", index_col=0)

In [23]:
df1 = pd.read_csv("i_result_high.csv", index_col=0)